<a href="https://colab.research.google.com/github/eagbdesign/lab-sql-generation-with-transformer-api/blob/main/lab-sql-generation-with-transformer-api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL Generation with Transformer API

In [1]:
!pip install torch transformers bitsandbytes accelerate sqlparse

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 8.7 MB/s eta 0:00:00


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [3]:
torch.cuda.is_available()

True

In [4]:
available_memory = torch.cuda.get_device_properties(0).total_memory

In [5]:
print(available_memory)

15835660288


##Download the Model
Use any model on Colab (or any system with >30GB VRAM on your own machine) to load this in f16. If unavailable, use a GPU with minimum 8GB VRAM to load this in 8bit, or with minimum 5GB of VRAM to load in 4bit.

This step can take around 5 minutes the first time. So please be patient :)

In [6]:
model_name = "defog/sqlcoder-7b-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if available_memory > 15e9:
    # if you have atleast 15GB of GPU memory, run load the model in float16
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        torch_dtype=torch.float16,
        device_map="auto",
        use_cache=True,
    )
else:
    # else, load in 8 bits – this is a bit slower
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        # torch_dtype=torch.float16,
        load_in_8bit=True,
        device_map="auto",
        use_cache=True,
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/515 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

##Set the Question & Prompt and Tokenize
Feel free to change the schema in the prompt below to your own schema

In [7]:
prompt = """### Task
Generate a SQL query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'
- Remember that revenue is price multiplied by quantity
- Remember that cost is supply_price multiplied by quantity

### Database Schema
This query will run on a database whose schema is represented in this string:
CREATE TABLE products (
  product_id INTEGER PRIMARY KEY, -- Unique ID for each product
  name VARCHAR(50), -- Name of the product
  price DECIMAL(10,2), -- Price of each unit of the product
  quantity INTEGER  -- Current quantity in stock
);

CREATE TABLE customers (
   customer_id INTEGER PRIMARY KEY, -- Unique ID for each customer
   name VARCHAR(50), -- Name of the customer
   address VARCHAR(100) -- Mailing address of the customer
);

CREATE TABLE salespeople (
  salesperson_id INTEGER PRIMARY KEY, -- Unique ID for each salesperson
  name VARCHAR(50), -- Name of the salesperson
  region VARCHAR(50) -- Geographic sales region
);

CREATE TABLE sales (
  sale_id INTEGER PRIMARY KEY, -- Unique ID for each sale
  product_id INTEGER, -- ID of product sold
  customer_id INTEGER,  -- ID of customer who made purchase
  salesperson_id INTEGER, -- ID of salesperson who made the sale
  sale_date DATE, -- Date the sale occurred
  quantity INTEGER -- Quantity of product sold
);

CREATE TABLE product_suppliers (
  supplier_id INTEGER PRIMARY KEY, -- Unique ID for each supplier
  product_id INTEGER, -- Product ID supplied
  supply_price DECIMAL(10,2) -- Unit price charged by supplier
);

-- sales.product_id can be joined with products.product_id
-- sales.customer_id can be joined with customers.customer_id
-- sales.salesperson_id can be joined with salespeople.salesperson_id
-- product_suppliers.product_id can be joined with products.product_id

### Answer
Given the database schema, here is the SQL query that answers [QUESTION]{question}[/QUESTION]
[SQL]
"""

##Generate the SQL
This can be excruciatingly slow on a T4 in Colab, and can take 10-20 seconds per query. On faster GPUs, this will take ~1-2 seconds

Ideally, you should use `num_beams`=4 for best results. But because of memory constraints, we will stick to just 1 for now.

In [8]:
import sqlparse

def generate_query(question):
    updated_prompt = prompt.format(question=question)
    inputs = tokenizer(updated_prompt, return_tensors="pt").to("cuda")
    generated_ids = model.generate(
        **inputs,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=400,
        do_sample=False,
        num_beams=1,
    )
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    # empty cache so that you do generate more results w/o memory crashing
    # particularly important on Colab – memory management is much more straightforward
    # when running on an inference service
    return sqlparse.format(outputs[0].split("[SQL]")[-1], reindent=True)

In [9]:
question = "What was our revenue by product in the New York region last month?"
generated_sql = generate_query(question)

In [10]:
print(generated_sql)


SELECT p.product_id,
       SUM(s.quantity * p.price) AS revenue
FROM sales s
JOIN salespeople sp ON s.salesperson_id = sp.salesperson_id
JOIN products p ON s.product_id = p.product_id
WHERE sp.region = 'New York'
  AND s.sale_date >= (CURRENT_DATE - INTERVAL '1 month')
GROUP BY p.product_id
ORDER BY revenue DESC NULLS LAST;


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

Report on Prompt Generation for SQL Queries with GPT
Summary of Exercises and Findings

In this exercise, we explored the capabilities of a GPT-based language model to generate SQL queries from natural language prompts. We created three distinct prompts and evaluated the model’s ability to produce accurate SQL outputs. Below are the three examples we tested:

Revenue and Sales Analysis for New York Region:

Prompt: "How many sales were made in Brazil, and how could we improve sales if we reduced prices?"
Database Schema: Included a sales table with details on quantity, region, and price per unit. The rules required calculating the total number of sales in Brazil and estimating the impact of a hypothetical 10% price reduction.
Results: The model generated accurate SQL queries using conditional clauses and group-by statements. It produced both the count of sales and a projection of revenues after a price reduction.
Elections in Venezuela:

Prompt: "Which political party received the most votes in each state during the 2023 Venezuelan elections?"
Database Schema: Included elections, candidates, and parties tables with data on votes, candidates, and political affiliations. The query required aggregating votes and finding the party with the maximum votes in each state.
Results: The model generated a multi-step SQL query using subqueries and CTEs (Common Table Expressions). The output was highly accurate and directly answered the question.
Exploration of Unstructured Schema:

Prompt: "List all regions where sales increased by more than 20% compared to the previous month."
Database Schema: Included sales and regions tables. This example introduced a new dimension of comparing data month-over-month.
Results: The model struggled to generate a correct query. It "hallucinated" non-existent columns like month and percentage_change, despite these not being defined in the schema. The output required significant manual correction to be usable.
Observations on Variations and Accuracy

Successful Outputs:

When the schema was detailed and well-structured, the model performed excellently, producing logical and syntactically correct SQL queries. Examples 1 and 2 demonstrated the importance of providing clear rules and a well-defined schema.
Challenges and Failures:

In the third prompt, where a dynamic calculation (month-over-month percentage) was expected but not explicitly defined in the schema, the model introduced hallucinated fields. This highlights the model’s tendency to invent columns or relationships that do not exist when the task is ambiguous.
Additionally, without detailed prompts, the model sometimes produced overly simplified SQL queries that did not fully meet the requirements.
What I Learned

Prompt Clarity is Crucial:

The more detailed the schema and instructions, the better the model performs. Ambiguity in prompts can lead to hallucinated or incorrect SQL output.
Complex Queries Require Explicit Definitions:

Tasks involving calculations or comparisons not directly represented in the schema need to be explicitly defined to avoid errors.
Potential of GPT in SQL Query Automation:

With appropriate guidance, GPT can automate SQL generation effectively. However, the outputs must always be validated by a human, especially in complex scenarios.
In conclusion, while GPT shows great promise in automating SQL tasks, its limitations must be carefully managed through precise and well-structured prompts.

In [11]:
# Define el esquema del prompt
prompt = """### Task
Generate a SQL query to answer the following question:
[QUESTION]How many sales were made in Brazil, and how could we improve sales if we reduced prices?[/QUESTION]

### Database Schema
You are working with the following database schema:

**Table: sales**
- sale_id (integer): Unique identifier for each sale.
- product_id (integer): Identifier of the product sold.
- region (text): Region where the sale occurred (e.g., 'Brazil', 'USA').
- sale_date (date): Date when the sale was made.
- quantity (integer): Number of units sold.
- price_per_unit (decimal): Price of each unit sold.
- revenue (decimal): Total revenue for this sale.

**Table: products**
- product_id (integer): Unique identifier for each product.
- product_name (text): Name of the product.
- category (text): Category to which the product belongs.

### Rules
1. Calculate the total number of sales (quantity) made in Brazil.
2. Suggest how sales could improve if prices were reduced by calculating the impact of a hypothetical price reduction (e.g., 10% less).
3. Use JOINs if necessary to fetch product names or categories.

[SQL]
"""

# Define la pregunta específica
question = "How many sales were made in Brazil, and how could we improve sales if we reduced prices?"

# Integra la pregunta con el prompt
updated_prompt = prompt.format(question=question)

# Tokeniza el prompt actualizado
inputs = tokenizer(updated_prompt, return_tensors="pt").to("cuda")

# Genera la consulta SQL usando el modelo
generated_ids = model.generate(
    **inputs,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    max_new_tokens=400,
    do_sample=False,
    num_beams=1,
)

# Decodifica la respuesta generada
outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

# Limpia y formatea el SQL generado
import sqlparse
generated_sql = sqlparse.format(outputs[0].split("[SQL]")[-1], reindent=True)

# Imprime la consulta SQL generada
print(generated_sql)



SELECT SUM(s.quantity) AS total_sales_in_brazil
FROM sales s
WHERE s.region = 'Brazil';


### Task
Generate a SQL query to answer the following question:
[QUESTION]Which political party received the most votes in each state during the 2023 Venezuelan elections?[/QUESTION]

### Database Schema
You are working with the following database schema:

**Table: elections**
- vote_id (integer): Unique identifier for each vote.
- state (text): The state in Venezuela where the vote was cast.
- municipality (text): The municipality where the vote was cast.
- candidate_id (integer): Identifier of the candidate voted for.
- party_id (integer): Identifier of the political party associated with the vote.
- election_date (date): Date of the election.

**Table: candidates**
- candidate_id (integer): Unique identifier for each candidate.
- candidate_name (text): Full name of the candidate.
- party_id (integer): Identifier of the political party the candidate represents.

**Table: parties**
- party_id (integer): Unique identifier for each political party.
- party_name (text): Name of the political party.

### Rules
1. Calculate the total number of votes received by each political party in each state.
2. Identify which political party received the highest number of votes in each state.
3. Use JOINs to fetch the party names from the `parties` table if necessary.

[SQL]


In [13]:
# Define el prompt
prompt = """### Task
Generate a SQL query to answer the following question:
[QUESTION]Which political party received the most votes in each state during the 2023 Venezuelan elections?[/QUESTION]

### Database Schema
You are working with the following database schema:

**Table: elections**
- vote_id (integer): Unique identifier for each vote.
- state (text): The state in Venezuela where the vote was cast.
- municipality (text): The municipality where the vote was cast.
- candidate_id (integer): Identifier of the candidate voted for.
- party_id (integer): Identifier of the political party associated with the vote.
- election_date (date): Date of the election.

**Table: candidates**
- candidate_id (integer): Unique identifier for each candidate.
- candidate_name (text): Full name of the candidate.
- party_id (integer): Identifier of the political party the candidate represents.

**Table: parties**
- party_id (integer): Unique identifier for each political party.
- party_name (text): Name of the political party.

### Rules
1. Calculate the total number of votes received by each political party in each state.
2. Identify which political party received the highest number of votes in each state.
3. Use JOINs to fetch the party names from the `parties` table if necessary.

[SQL]
"""

# Define la pregunta específica
question = "Which political party received the most votes in each state during the 2023 Venezuelan elections?"

# Integra la pregunta con el prompt
updated_prompt = prompt.format(question=question)

# Tokeniza el prompt actualizado
inputs = tokenizer(updated_prompt, return_tensors="pt").to("cuda")

# Genera la consulta SQL usando el modelo
generated_ids = model.generate(
    **inputs,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    max_new_tokens=400,
    do_sample=False,
    num_beams=1,
)

# Decodifica la respuesta generada
outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

# Limpia y formatea el SQL generado
import sqlparse
generated_sql = sqlparse.format(outputs[0].split("[SQL]")[-1], reindent=True)

# Imprime la consulta SQL generada
print(generated_sql)



SELECT e.state,
       p.party_name,
       SUM(e.votes) AS total_votes
FROM elections e
JOIN parties p ON e.party_id = p.party_id
GROUP BY e.state,
         p.party_name
ORDER BY total_votes DESC;
